In [1]:
import findspark
findspark.init('/opt/spark/2.4.4/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.\
        builder.\
        appName('NULL_CHECK'). \
        enableHiveSupport(). \
        getOrCreate()
sc = spark.sparkContext

### Get all transaction ids where there is atlest one null value in row

In [132]:
df = spark.read.csv('file:///home/rameshbabug/Documents/projects/internal/spark-playground/src/data/transaction.csv', header=True)
df.show(5)

+----------+-----------+-----+--------------+
|      Date|Transaction| Name|Transaction_ID|
+----------+-----------+-----+--------------+
|29-10-2020|       null|“ABC”|          tid1|
|29-10-2020|        124|“BCD”|          tid2|
|30-10-2020|        125| null|          tid3|
|30-10-2020|        125| null|          tid3|
|      null|       null| null|          null|
+----------+-----------+-----+--------------+
only showing top 5 rows



In [133]:
df.count()

16

#### Drop na records i.e. get all not null records

In [134]:
df_new = df.dropna()
df_new.select(df['Transaction_ID']).show(5)

+--------------+
|Transaction_ID|
+--------------+
|          tid2|
|          tid4|
|          tid6|
|          tid7|
|          tid9|
+--------------+
only showing top 5 rows



In [135]:
df_new.count()

8

#### Get records with nulls

In [136]:
from pyspark.sql.functions import col

In [137]:
# df_null_rows = 
df = df.selectExpr("Date as df1_date", "Transaction as df1_Transaction", "Name as df1_Name", "Transaction_ID as df1_Transaction_ID")
df.printSchema()

root
 |-- df1_date: string (nullable = true)
 |-- df1_Transaction: string (nullable = true)
 |-- df1_Name: string (nullable = true)
 |-- df1_Transaction_ID: string (nullable = true)



In [138]:
# df_null_rows = 
df_new = df_new.selectExpr("Date as df2_date", "Transaction as df2_Transaction", "Name as df2_Name", "Transaction_ID as df2_Transaction_ID")
df_new.printSchema()

root
 |-- df2_date: string (nullable = true)
 |-- df2_Transaction: string (nullable = true)
 |-- df2_Name: string (nullable = true)
 |-- df2_Transaction_ID: string (nullable = true)



In [139]:
df_null_rows = df.join(df_new.alias('dn'),df['df1_Transaction_ID'] == df_new['df2_TRansaction_ID'], 'left')

In [140]:
df_null_rows.show(5)

+----------+---------------+--------+------------------+----------+---------------+--------+------------------+
|  df1_date|df1_Transaction|df1_Name|df1_Transaction_ID|  df2_date|df2_Transaction|df2_Name|df2_Transaction_ID|
+----------+---------------+--------+------------------+----------+---------------+--------+------------------+
|29-10-2020|           null|   “ABC”|              tid1|      null|           null|    null|              null|
|29-10-2020|            124|   “BCD”|              tid2|29-10-2020|            124|   “BCD”|              tid2|
|30-10-2020|            125|    null|              tid3|      null|           null|    null|              null|
|30-10-2020|            125|    null|              tid3|      null|           null|    null|              null|
|      null|           null|    null|              null|      null|           null|    null|              null|
+----------+---------------+--------+------------------+----------+---------------+--------+------------

In [141]:
df_null_rows = df_null_rows.where('df2_Transaction_ID is null').select('df1_Transaction_ID').show()

+------------------+
|df1_Transaction_ID|
+------------------+
|              tid1|
|              tid3|
|              tid3|
|              null|
|              tid5|
|              tid8|
|             tid10|
|             tid12|
+------------------+



#### Alternative way i.e. using Spark SQL

In [79]:
df.show()
df.createOrReplaceTempView('original_df')

+----------+-----------+-----+--------------+
|      Date|Transaction| Name|Transaction_ID|
+----------+-----------+-----+--------------+
|29-10-2020|       null|“ABC”|          tid1|
|29-10-2020|        124|“BCD”|          tid2|
|30-10-2020|        125| null|          tid3|
|30-10-2020|        125| null|          tid3|
|      null|       null| null|          null|
|30-10-2020|        126|“ABC”|          tid4|
|      null|       null|“ABC”|          tid5|
|30-10-2020|        128|“ABC”|          tid6|
|30-10-2020|        129|“XYZ”|          tid7|
|      null|        130| null|          tid8|
|30-10-2020|        131|“ABC”|          tid9|
|30-10-2020|       null|“ABC”|         tid10|
|30-10-2020|        133|“ABC”|         tid11|
|      null|       null|“XYZ”|         tid12|
|30-10-2020|        135|“XYZ”|         tid13|
|30-10-2020|        136|“XYZ”|         tid14|
+----------+-----------+-----+--------------+



In [80]:
df_new.show()
df_new.createOrReplaceTempView('not_nulls_df')

+----------+-----------+-----+--------------+
|      Date|Transaction| Name|Transaction_ID|
+----------+-----------+-----+--------------+
|29-10-2020|        124|“BCD”|          tid2|
|30-10-2020|        126|“ABC”|          tid4|
|30-10-2020|        128|“ABC”|          tid6|
|30-10-2020|        129|“XYZ”|          tid7|
|30-10-2020|        131|“ABC”|          tid9|
|30-10-2020|        133|“ABC”|         tid11|
|30-10-2020|        135|“XYZ”|         tid13|
|30-10-2020|        136|“XYZ”|         tid14|
+----------+-----------+-----+--------------+



In [87]:
df_null_rows = spark.sql('''select d.Transaction_ID from original_df d 
left join not_nulls_df dn 
on d.Transaction_ID= dn.Transaction_ID
where dn.Transaction_ID is null''')

In [88]:
df_null_rows.show()

+--------------+
|Transaction_ID|
+--------------+
|          tid1|
|          tid3|
|          tid3|
|          null|
|          tid5|
|          tid8|
|         tid10|
|         tid12|
+--------------+



### Drop Duplicate Records

In [142]:
df = spark.read.csv('file:///home/rameshbabug/Documents/projects/internal/spark-playground/src/data/transaction.csv', header=True)
df.show(5)

+----------+-----------+-----+--------------+
|      Date|Transaction| Name|Transaction_ID|
+----------+-----------+-----+--------------+
|29-10-2020|       null|“ABC”|          tid1|
|29-10-2020|        124|“BCD”|          tid2|
|30-10-2020|        125| null|          tid3|
|30-10-2020|        125| null|          tid3|
|      null|       null| null|          null|
+----------+-----------+-----+--------------+
only showing top 5 rows



In [143]:
df.count()

16

In [144]:
df= df.drop_duplicates()
df.show(5)

+----------+-----------+-----+--------------+
|      Date|Transaction| Name|Transaction_ID|
+----------+-----------+-----+--------------+
|      null|       null|“ABC”|          tid5|
|      null|       null| null|          null|
|29-10-2020|        124|“BCD”|          tid2|
|      null|        130| null|          tid8|
|30-10-2020|       null|“ABC”|         tid10|
+----------+-----------+-----+--------------+
only showing top 5 rows



In [145]:
df.count()

15

### Add columns not present in df2 into df1

In [185]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Transaction: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Transaction_ID: string (nullable = true)



In [186]:
df_new = df.dropna()
df_new = df_new.selectExpr("Date as Date1", "Transaction", "Name as Name1", "Transaction_ID")
df_new.printSchema()

root
 |-- Date1: string (nullable = true)
 |-- Transaction: string (nullable = true)
 |-- Name1: string (nullable = true)
 |-- Transaction_ID: string (nullable = true)



#### Here Date1 and Name2 not present in df1, add those

In [187]:
columns1 = df.columns
columns1

['Date', 'Transaction', 'Name', 'Transaction_ID']

In [188]:
columns2 = df_new.columns
columns2

['Date1', 'Transaction', 'Name1', 'Transaction_ID']

In [189]:
updated_columns1 = []
for col in columns1:
    updated_columns1.append('d.'+col)
updated_columns1

['d.Date', 'd.Transaction', 'd.Name', 'd.Transaction_ID']

In [192]:
new_cols = [i for i in columns1 + columns2 if i not in columns1]
new_cols.extend(updated_columns1)
new_cols

['Date1', 'Name1', 'd.Date', 'd.Transaction', 'd.Name', 'd.Transaction_ID']

In [193]:
df_updated = df.alias('d').join(df_new.alias('d1'), 'Transaction_ID', 'left').select(new_cols)

In [194]:
df_updated.show(5)

+----------+-----+----------+-----------+-----+--------------+
|     Date1|Name1|      Date|Transaction| Name|Transaction_ID|
+----------+-----+----------+-----------+-----+--------------+
|      null| null|      null|       null|“ABC”|          tid5|
|      null| null|      null|       null| null|          null|
|29-10-2020|“BCD”|29-10-2020|        124|“BCD”|          tid2|
|      null| null|      null|        130| null|          tid8|
|      null| null|30-10-2020|       null|“ABC”|         tid10|
+----------+-----+----------+-----------+-----+--------------+
only showing top 5 rows

